# Summary
## 1. Train data exploration & cleaning
## 2. XGBoost classification
## 3. Test dataset cleaning
## 4. Final prediction & Kaggle score
## 5. Further improvement

In [164]:
# Relevant libraries
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split

# pip3 install xgboost==0.80, (version 0.90 crashes)
import xgboost as xgb

import copy

# 1. Train data exploration & cleaning

In [165]:
# Titanic Dataset
# https://www.kaggle.com/c/titanic/data
# Training dataset
train = pd.read_csv('titanic_train.csv')

#Testing dataset
test = pd.read_csv('titanic_test.csv')

In [166]:
## Data Exploration
# Check missing values
train.isna().sum()
#There are 177 missing 'Age' values, 657 missing 'Cabin' values and 2  missing 'Embarked' values

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [167]:
# Descriptive statistics
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [168]:
# Data Exploration
# Feature : SibSp
# train.SibSp.hist()
# len(train.SibSp[train.SibSp<=1])/len(train.SibSp)
# The historgram of SibSp feature shows a high imbalance with up to 92% of the samples between 0 and 1 and 8% in 5 other classes.
# The minority of 8% of samples will be merged and labelled with 2 to balance the dataset and improve the final model's 
# accuracy on unseen data which may belong to this minority. 

def SibSp(row):
    if row['SibSp']<=1:
        result = row['SibSp']
    else: # Group samples with 2 and above in a single bin
        result = 2
    return result

train['SibSp'] = train.apply(lambda row:SibSp(row), axis=1)

In [169]:
# Data Exploration
# Feature : Name
# Split Names to extract titles
train['title'] = train.Name.str.split(", ", n = 1, expand = True)[1].str.split(".", n =1, expand=True).iloc[:,0]

# List titles
# train.title.value_counts()

# Replace titles with labels 0,1, 2, 3
map_name = {'Master':0, 'Mr':0, 'Mlle':1, 'Ms':1, 'Miss':1, 'Mme':2, 'Mrs':2}
train['title'] = train['title'].apply(lambda n:map_name[n] if n in map_name else 3)

# Drop names
train = train.drop(columns = ['Name'])

train.title.value_counts()

0    557
1    185
2    126
3     23
Name: title, dtype: int64

In [170]:
# Data Exploration
# Feature : Parch
# train.Parch.hist()
# train.Parch.value_counts()
# len(train.Parch[train.Parch<=1])/len(train.Parch)
# 89% of the samples between 0 and 1 and 11% in 5 other classes.
# This minority of 11% of samples are merged into a single bin labelled 2.

def Parch(row):
    if row['Parch']<=1:
        result = row['Parch']
    else: # Group samples with 2 and above in a single bin
        result = 2
    return result

train['Parch'] = train.apply(lambda row:Parch(row), axis=1)
train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,title
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,2
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S,2
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,211536,13.0000,NaN,S,3
887,888,1,1,female,19.0,0,0,112053,30.0000,B42,S,1
888,889,0,3,female,NaN,1,2,W./C. 6607,23.4500,NaN,S,1
889,890,1,1,male,26.0,0,0,111369,30.0000,C148,C,0


In [171]:
## Data Exploration
# Feature : Cabin
# Set 0 for no Cabin information and 1 for Cabin information
train['Cabin'] = train['Cabin'].apply(lambda x: 0 if type(x) == float else 1)
train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,title
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,0,S,0
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,1,C,2
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,0,S,1
3,4,1,1,female,35.0,1,0,113803,53.1000,1,S,2
4,5,0,3,male,35.0,0,0,373450,8.0500,0,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,211536,13.0000,0,S,3
887,888,1,1,female,19.0,0,0,112053,30.0000,1,S,1
888,889,0,3,female,NaN,1,2,W./C. 6607,23.4500,0,S,1
889,890,1,1,male,26.0,0,0,111369,30.0000,1,C,0


In [172]:
## Data Exploration
# PassengerId and Ticket dropped as they are user-specific
train = train.drop(columns = ['PassengerId', 'Ticket'])
train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,title
0,0,3,male,22.0,1,0,7.2500,0,S,0
1,1,1,female,38.0,1,0,71.2833,1,C,2
2,1,3,female,26.0,0,0,7.9250,0,S,1
3,1,1,female,35.0,1,0,53.1000,1,S,2
4,0,3,male,35.0,0,0,8.0500,0,S,0
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,0,S,3
887,1,1,female,19.0,0,0,30.0000,1,S,1
888,0,3,female,NaN,1,2,23.4500,0,S,1
889,1,1,male,26.0,0,0,30.0000,1,C,0


In [173]:
## Data Exploration
# Features : Embarked, Sex
# Set the 2 missing Embarked values as S (most frequent label)
train['Embarked'] = train['Embarked'].fillna('S')

# Convert 'Sex' and 'Embarked' to numerical labels
# Display unique values of Sex and Embarked : train['Sex'].unique(), train['Embarked'].unique()
map_s = {'male':1, 'female':0}
map_e = {'Q':0, 'C':1, 'S':2}

train.loc[:,'Sex'] = train.loc[:,'Sex'].map(lambda s:map_s[s])
train.loc[:,'Embarked'] = train.loc[:,'Embarked'].map(lambda e:map_e[e])

train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,title
0,0,3,1,22.0,1,0,7.2500,0,2,0
1,1,1,0,38.0,1,0,71.2833,1,1,2
2,1,3,0,26.0,0,0,7.9250,0,2,1
3,1,1,0,35.0,1,0,53.1000,1,2,2
4,0,3,1,35.0,0,0,8.0500,0,2,0
...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000,0,2,3
887,1,1,0,19.0,0,0,30.0000,1,2,1
888,0,3,0,NaN,1,2,23.4500,0,2,1
889,1,1,1,26.0,0,0,30.0000,1,1,0


In [175]:
## Data Exploration
# Feature : Fare.
# We group fares into bins to reduce imbalance and improve accuracy.
# Merge Fares into n bins
# pd.qcut(train['Fare'], q=2).value_counts()

def Fare(row):
    if row['Fare'] <= 14.454:     # Class 0 for Fares (0, 14.454]
        result = 0
    else:                         # Class 1 for Fares (14.454, 512.329]
        result = 1
    return result

train['Fare'] = train.apply(lambda row:Fare(row), axis=1)

train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,title
0,0,3,1,22.0,1,0,0,0,2,0
1,1,1,0,38.0,1,0,1,1,1,2
2,1,3,0,26.0,0,0,0,0,2,1
3,1,1,0,35.0,1,0,1,1,2,2
4,0,3,1,35.0,0,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,0,0,2,3
887,1,1,0,19.0,0,0,1,1,2,1
888,0,3,0,NaN,1,2,1,0,2,1
889,1,1,1,26.0,0,0,1,1,1,0


In [ ]:
## Data Exploration
# Feature : Age.
# 20% of Age values are missing and our intuition is that Age information may be useful in predicting survival.
# Several options for handling missing data exist. Some are :
# 1- Delete the samples/undersample the data : Not a good idea as we could loose much information in 20% of the dataset.
# 2- Find a model to predict the missing Ages using the samples with Age information.
# 3- Generate Ages using the mean and std of available Ages
# Option 3 is used here
# Mean and standard deviation of Ages
mean_age = train['Age'][~train['Age'].isna()].mean()
std_age = train['Age'][~train['Age'].isna()].std()

missing_age_count = train['Age'].isna().sum()

# Generate ages between mean-std & mean+std
generated_ages = np.random.randint(mean_age-std_age, mean_age+std_age, size=missing_age_count)

# Fill the missing ages in train dataset
train['Age'][train['Age'].isna()] = generated_ages
train

In [177]:
## Data Exploration
# Feature : Age.
# Split Ages into n bins

# Determine the number bins to be used, this can be modified to check the impact on model performance
#pd.qcut(train['Age'], q=5).unique()

def Age(row):
    if row['Age'] <= 19.0:     # Class 0 for Ages (0.419, 19.0]
        result = 0
    elif row['Age'] <= 25.0:   # Class 1 for Ages (19.0, 25.0]
        result = 1
    elif row['Age'] <= 32.0:   # Class 2 for Ages (25.0, 32.0]
        result = 2
    elif row['Age'] <= 40.0:   # Class 3 for Ages (32.0, 40.0]
        result = 3
    else:                      # Class 4 for Ages (40.0, 80.0]
        result = 4
    return result

train['Age'] = train.apply(lambda row:Age(row), axis=1)
train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,title
0,0,3,1,1,1,0,0,0,2,0
1,1,1,0,3,1,0,1,1,1,2
2,1,3,0,2,0,0,0,0,2,1
3,1,1,0,3,1,0,1,1,2,2
4,0,3,1,3,0,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,2,0,0,0,0,2,3
887,1,1,0,0,0,0,1,1,2,1
888,0,3,0,3,1,2,1,0,2,1
889,1,1,1,2,0,0,1,1,1,0


In [178]:
train.corr()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,title
Survived,1.000000,-0.338481,-0.543351,-0.024637,0.054203,0.121076,0.267214,0.316912,-0.106811,0.446144
Pclass,-0.338481,1.000000,0.131900,-0.274408,-0.000390,-0.010256,-0.537830,-0.725541,-0.045702,-0.242421
Sex,-0.543351,0.131900,1.000000,0.039968,-0.177623,-0.253248,-0.235243,-0.140391,0.116569,-0.760886
Age,-0.024637,-0.274408,0.039968,1.000000,-0.161943,-0.194706,0.072633,0.195709,-0.000861,0.143396
SibSp,0.054203,-0.000390,-0.177623,-0.161943,1.000000,0.450499,0.487085,0.022056,0.038850,0.168865
Parch,0.121076,-0.010256,-0.253248,-0.194706,0.450499,1.000000,0.430410,0.066414,0.083436,0.186591
Fare,0.267214,-0.537830,-0.235243,0.072633,0.487085,0.430410,1.000000,0.410011,-0.000044,0.282137
Cabin,0.316912,-0.725541,-0.140391,0.195709,0.022056,0.066414,0.410011,1.000000,-0.013774,0.171772
Embarked,-0.106811,-0.045702,0.116569,-0.000861,0.038850,0.083436,-0.000044,-0.013774,1.000000,-0.062275
title,0.446144,-0.242421,-0.760886,0.143396,0.168865,0.186591,0.282137,0.171772,-0.062275,1.000000


In [179]:
# X train and Y train
Xtr = train.drop(columns=['Survived'])
Ytr = train[['Survived']]

# 2. XGBoost Classification
### Four different classification algorithms are used at the first step on the training set. Next, the ensemble learning method - boosting is used to construct a final model.

In [188]:
# Run Classification with different algorithms
rf = RandomForestClassifier(n_estimators=500, n_jobs= -1,max_depth = 6, min_samples_leaf= 2, 
                            max_features='sqrt', verbose= 0)
et = ExtraTreesClassifier(n_jobs= -1, n_estimators=500, max_depth= 8, min_samples_leaf=2, verbose=0)
ada = AdaBoostClassifier(n_estimators= 500, learning_rate= 0.75)
gb = GradientBoostingClassifier(n_estimators=500, max_depth= 5, min_samples_leaf= 2, verbose=0)

# Fit the models
rf.fit(Xtr, Ytr.to_numpy().ravel())
et.fit(Xtr, Ytr.to_numpy().ravel())
ada.fit(Xtr, Ytr.to_numpy().ravel())
gb.fit(Xtr, Ytr.to_numpy().ravel())

# Use predictions of these classifiers as input for XGBoost
New_train = pd.DataFrame( {'RandomForest': rf.predict(Xtr).ravel(), 'ExtraTrees': et.predict(Xtr).ravel(),
                           'AdaBoost': ada.predict(Xtr).ravel(), 'GradientBoost': gb.predict(Xtr).ravel() })

# Split predictions into training and validation set to evaluate the XGBoost model
X_train, X_test, Y_train, Y_test = train_test_split(New_train, Ytr, test_size=0.2, random_state=20)

# Run XGBoost Classifier
xgbm = xgb.XGBClassifier(scale_pos_weight=0.9) #Using default parameters
xgbm.fit(X_train, Y_train.to_numpy().ravel())
xgbm.score(X_test, Y_test)

0.8715083798882681

# 3. Test dataset cleaning

In [181]:
test.isna().sum()
#There are 86 missing values of Age, we will predict the Age group for these samples as done earlier, before predicting survival
#There is 1 sample with no Fare value

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [ ]:
test_proc = copy.deepcopy(test)

#SibSp
test_proc['SibSp'] = test_proc.apply(lambda row:SibSp(row), axis=1)


#Name/title
test_proc['title'] = test_proc.Name.str.split(", ", n = 1, expand = True)[1].str.split(".", n =1, expand=True).iloc[:,0]
test_proc['title'] = test_proc['title'].apply(lambda n:map_name[n] if n in map_name else 3)
test_proc = test_proc.drop(columns = ['Name'])


#Parch
test_proc['Parch'] = test_proc.apply(lambda row:Parch(row), axis=1)
test_proc['Cabin'] = test_proc['Cabin'].apply(lambda x: 0 if type(x) == float else 1)


#Embarked
test_proc['Embarked'] = test_proc['Embarked'].fillna('S')
test_proc.loc[:,'Embarked'] = test_proc.loc[:,'Embarked'].map(lambda e:map_e[e])


#Sex
map_s = {'male':1, 'female':0}
test_proc.loc[:,'Sex'] = test_proc.loc[:,'Sex'].map(lambda s:map_s[s])


#Fare
test_proc['Fare'] = test_proc.apply(lambda row:Fare(row), axis=1)


#Age
mean_age = test_proc['Age'][~test_proc['Age'].isna()].mean()
std_age = test_proc['Age'][~test_proc['Age'].isna()].std()
missing_age_count = test_proc['Age'].isna().sum()
generated_ages = np.random.randint(mean_age-std_age, mean_age+std_age, size=missing_age_count)
test_proc['Age'][test_proc['Age'].isna()] = generated_ages
test_proc['Age'] = test_proc.apply(lambda row:Age(row), axis=1)


test_proc = test_proc.drop(columns = ['PassengerId', 'Ticket'])

test_proc

In [183]:
# Fit final XGBoost classifier on full training dataset
xgbm.fit(New_train, Ytr.to_numpy().ravel())
xgbm

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
              max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
              n_jobs=1, nthread=None, objective='binary:logistic',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=True, subsample=1)

# 4. Final Prediction

In [ ]:
# Predict Survival 
# First run the first level classifiers on processed test data : test_proc
New_test = pd.DataFrame( {'RandomForest': rf.predict(test_proc).ravel(), 'ExtraTrees': et.predict(test_proc).ravel(),
                           'AdaBoost': ada.predict(test_proc).ravel(), 'GradientBoost': gb.predict(test_proc).ravel() })

# Run XGBoost Classifier on previous result
Y_predict = xgbm.predict(New_test)

# Put prediction in a df
Survival_pred = pd.DataFrame(Y_predict, columns=['Survived'])

# Pick PassengerIds from test df
Survival_submission = test[['PassengerId']]

# Reset indices of dfs to be combined
Survival_pred.index = Survival_submission.index

# Final dataframe
Survival_submission[['Survived']] = Survival_pred

# Save to csv
Survival_submission.to_csv('submissionxxx.csv', sep=',', mode='w', header=True, index=False)

#### Kaggle score for this submission : 0.77033 (100th rank)

# 5. Further Improvement
#### This is a usually a time-consuming exercise which does not guarantee significant improvement in model performance.
#### In order to improve the performance of this model, we could try for instance :
#### - Early stopping : Iteratively run XGBoost on a higher number of trees and stop when the validation score remains constant
####                              over a specified number of iterations. A possible evaluation metric is the area under the ROC curve (AUC).
#### - Tuning other hyperparameters/training a neural network

In [197]:
# Early stopping example
xgbm_tun = xgb.XGBClassifier(n_estimators=10000)

X_train, X_test, Y_train, Y_test = train_test_split(New_train, Ytr, test_size=0.2, random_state=20)

# Evaluating on New_train (output of 4 classifiers)
eval_set  = [(X_train,Y_train), (X_test,Y_test)]
xgbm_tun.fit(X_train, Y_train, eval_set=eval_set, eval_metric="auc", early_stopping_rounds=30)
# This model gave the same Kaggle score as the previous one : 0.77033

[0]	validation_0-auc:0.894221	validation_1-auc:0.852212
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 30 rounds.
[1]	validation_0-auc:0.894221	validation_1-auc:0.852212
[2]	validation_0-auc:0.894221	validation_1-auc:0.852212
[3]	validation_0-auc:0.894221	validation_1-auc:0.852212
[4]	validation_0-auc:0.894221	validation_1-auc:0.852212
[5]	validation_0-auc:0.894221	validation_1-auc:0.852212
[6]	validation_0-auc:0.894221	validation_1-auc:0.852212
[7]	validation_0-auc:0.894221	validation_1-auc:0.852212
[8]	validation_0-auc:0.894221	validation_1-auc:0.852212
[9]	validation_0-auc:0.894221	validation_1-auc:0.852212
[10]	validation_0-auc:0.897982	validation_1-auc:0.8748
[11]	validation_0-auc:0.898382	validation_1-auc:0.8752
[12]	validation_0-auc:0.898431	validation_1-auc:0.8752
[13]	validation_0-auc:0.898431	validation_1-auc:0.8752
[14]	validation_0-auc:0.898398	validation_1-auc:0.8752
[15]	val

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
              max_depth=3, min_child_weight=1, missing=None, n_estimators=10000,
              n_jobs=1, nthread=None, objective='binary:logistic',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=True, subsample=1)